In [9]:
import pandas as pd
import os
import time
import lightgbm as lgb
import matplotlib.pyplot as plt
import warnings
from datetime import datetime
from bayes_opt import BayesianOptimization
from lightgbm import LGBMClassifier
import numpy as np

import settings_model

warnings.simplefilter(action='ignore', category=FutureWarning)

pd.set_option('display.max_rows', 50000)
pd.set_option('display.max_columns', 50000)

print('lightGBM version', lgb.__version__)

# model_id = "HUVEC_20190810_202545"
model_id = "HEPG2_20190811_142600"
output_dir = os.path.join(settings_model.root_path, "models", "siamese-cell",
                          f"{model_id}", "emb")

lightGBM version 2.2.3


In [5]:
df_train = pd.read_csv(os.path.join(output_dir, "emb_train.csv"), header=None)#.sample(frac=0.5)
df_valid = pd.read_csv(os.path.join(output_dir, "emb_valid.csv"), header=None)
print("train", df_train.shape)
print("valid", df_valid.shape)
df_train.columns = df_train.columns.tolist()[:-1] + ["TARGET"]
df_valid.columns = df_valid.columns.tolist()[:-1] + ["TARGET"]
df_train.head()

train (86220, 129)
valid (992, 129)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,TARGET
22570,1.115120,-0.812163,0.999520,0.473936,-0.111045,0.401779,0.298741,0.027764,-0.198880,0.195782,-0.203637,1.481260,0.037662,1.206008,0.135087,-0.701905,0.401269,0.770378,-0.384910,1.051266,-0.456784,-0.002182,0.541055,0.248358,-1.100184,-0.175203,-0.208046,-0.867965,0.499489,0.186703,0.184205,0.983467,0.171228,-0.473962,-0.067935,-0.746544,1.021289,1.183232,0.386953,-0.641643,0.311075,-0.683508,0.151159,-0.835884,0.314692,0.222327,-0.951957,-0.330503,-0.954284,-0.907088,-0.381297,-0.568039,0.165597,0.070025,0.762621,0.761588,0.041996,-0.491939,0.266267,-0.043106,0.338545,-0.468541,-0.267147,0.514451,-0.990372,0.559909,0.069218,0.906935,-0.240570,-0.102345,0.305328,0.917522,1.085336,-0.495803,0.947908,-0.769751,-1.088021,0.469260,0.305072,-0.641360,0.150511,0.068960,1.043315,-0.418300,1.085144,-0.928959,-0.167872,0.812571,-0.183837,-0.667301,0.518967,-0.783982,0.553192,0.482868,0.470178,0.178276,0.539762,0.035636,-0.467180,0.499896,-0.580248,0.625731,0.836909,-0.334144,0.471594,0.621171,-0.150757,-0.340110,0.281607,-0.465407,-0.528346,0.132581,-0.491218,0.739732,0.371598,-0.109128,-0.263545,0.003661,0.851574,0.607503,0.562765,0.457904,0.179819,-0.511334,-0.520777,0.024689,-0.189900,-0.254532,842
16685,0.942946,-0.648891,0.856448,0.417749,-0.028715,0.315346,0.253220,0.029043,-0.127225,0.138472,-0.150499,1.254624,0.063846,1.008367,0.133532,-0.587394,0.306522,0.590236,-0.309718,0.814211,-0.454224,-0.044430,0.485063,0.254149,-0.977464,-0.069657,-0.162968,-0.706081,0.385089,0.159863,0.103334,0.869885,0.189985,-0.437686,-0.103969,-0.707402,0.890293,1.008038,0.294505,-0.596239,0.295053,-0.581890,0.074221,-0.689581,0.240678,0.243086,-0.774149,-0.269381,-0.845396,-0.747802,-0.357692,-0.575798,0.075748,0.053659,0.603290,0.588036,-0.044867,-0.231689,0.222319,-0.028392,0.322678,-0.388349,-0.233056,0.504491,-0.827719,0.423305,0.022377,0.745191,-0.158106,-0.051284,0.236679,0.785260,0.909442,-0.423034,0.779821,-0.655471,-0.895765,0.324838,0.283415,-0.495799,0.173841,0.071632,0.870188,-0.370303,0.857812,-0.800465,-0.098202,0.706219,-0.139726,-0.653560,0.447379,-0.664467,0.430281,0.468289,0.429375,0.127178,0.524980,-0.075853,-0.353152,0.462548,-0.459914,0.565993,0.650294,-0.299079,0.456827,0.524460,-0.140988,-0.250238,0.290855,-0.331692,-0.456499,0.110501,-0.394977,0.630778,0.327174,-0.140782,-0.256002,0.000502,0.696451,0.415953,0.457629,0.395805,0.172674,-0.452176,-0.466752,0.021225,-0.216684,-0.218758,355
123440,1.121289,-0.803799,1.045861,0.504945,-0.051864,0.485084,0.399641,0.072972,-0.139932,0.116327,-0.175476,1.567504,0.113902,1.208327,0.097454,-0.642899,0.388071,0.740143,-0.353415,0.988833,-0.552584,0.008696,0.519598,0.220283,-1.153672,-0.086348,-0.296566,-0.891800,0.575132,0.156740,0.151445,0.947815,0.176358,-0.513119,-0.141243,-0.855411,1.111049,1.181610,0.266485,-0.782169,0.305061,-0.756833,0.117234,-0.832389,0.225036,0.380719,-0.864756,-0.381530,-0.909695,-0.860609,-0.492003,-0.759491,0.094117,0.057793,0.702132,0.687047,-0.006143,-0.398009,0.303515,0.107709,0.446104,-0.495050,-0.243958,0.582464,-1.070652,0.549145,0.059315,0.868515,-0.249779,-0.150307,0.288268,0.953198,1.076480,-0.557308,1.064692,-0.803946,-1.160680,0.444875,0.233066,-0.585349,0.186286,0.070056,1.118312,-0.422528,0.969558,-1.010211,-0.098995,0.764818,-0.184648,-0.706490,0.568750,-0.762300,0.539530,0.464918,0.484420,0.236719,0.562145,-0.236458,-0.464758,0.662213,-0.561075,0.655185,0.800641,-0.334941,0.421006,0.629312,-0.174035,-0.345552,0.307037,-0.442606,-0.579471,0.117530,-0.487585,0.723310,0.459936,-0.063990,-0.267776,0.000693,0.861442,0.550744,0.593329,0.40633

In [6]:
features = df_train.columns.tolist()[:-1]
n_classes = df_train["TARGET"].nunique()
labels_valid = df_valid["TARGET"].unique()

In [7]:
class color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'

In [ ]:
def accuracy(preds, train_data):
    labels = train_data.get_label()
    pred = np.argmax(preds.reshape(n_classes, len(preds)//n_classes), axis=0)
    return 'accuracy', np.mean(labels == pred), True

def lgbm_evaluate(**params):
    start = time.time()
    params['num_leaves'] = int(params['num_leaves'])
    params['max_depth'] = int(params['max_depth'])
        
    clf = LGBMClassifier(**params, 
                         n_estimators=10000, 
                         n_jobs=os.cpu_count(),
                         objective="multiclass",
                         num_class=n_classes,
                        )
        
    clf.fit(df_train[features].values, df_train["TARGET"].values, 
            eval_set = [(df_train[features].values, df_train["TARGET"].values),
                        (df_valid[features].values, df_valid["TARGET"].values)],
            early_stopping_rounds=5, verbose=1)
    
#     train_preds = clf.predict_proba(df_train[features].values, num_iteration=clf.best_iteration_)
    valid_preds = clf.predict_proba(df_valid[features].values, num_iteration=clf.best_iteration_)
    
#     print('Accuracy train {:.6f}'.format(sum(np.argmax(train_preds, axis=1) == df_train['TARGET'].values) / float(len(train_preds))))
    acc_valid = np.mean(np.argmax(valid_preds, axis=1) == df_valid['TARGET'].values)
        
    return acc_valid

def optimize_lgbm():
    
    params_space = {'colsample_bytree': (0.9, 1.0),
                    'learning_rate': (0.01, 1.0), 
                    'num_leaves': (20, 1000), 
                    'subsample': (0.5, 1.0), 
                    'max_depth': (2, 1000), 
                    'reg_alpha': (0.0, 1.0), 
                    'reg_lambda': (0.0, 1.0), 
                    'min_split_gain': (0.0001, 1.),
                    'min_child_weight': (5., 200.),
                   }

    bo = BayesianOptimization(lgbm_evaluate, params_space)
    bo.maximize(init_points = 50, n_iter = 50)
    
    best_acc = bo.max['target']
    best_params = bo.max['params']
    best_params['num_leaves'] = int(best_params['num_leaves'])
    best_params['max_depth'] = int(best_params['max_depth'])
    
    
    print("Best validation acc: {}".format(best_acc))
    print('Best parameters found by optimization:\n')
    for k, v in best_params.items():
        print(color.BLUE + k + color.END + ' = ' + color.BOLD + str(v)+ color.END + '     [',params_space[k],']')
        
    return best_acc, best_params

best_acc, best_params = optimize_lgbm()

|   iter    |  target   | colsam... | learni... | max_depth | min_ch... | min_sp... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------
[1]	valid_0's multi_logloss: 11.119	valid_1's multi_logloss: 3.23724
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's multi_logloss: 10.7781	valid_1's multi_logloss: 3.21482
[3]	valid_0's multi_logloss: 10.4394	valid_1's multi_logloss: 3.19642
[4]	valid_0's multi_logloss: 10.1033	valid_1's multi_logloss: 3.18325
[5]	valid_0's multi_logloss: 9.77052	valid_1's multi_logloss: 3.17309
[6]	valid_0's multi_logloss: 9.44253	valid_1's multi_logloss: 3.16966
[7]	valid_0's multi_logloss: 9.12086	valid_1's multi_logloss: 3.17322
[8]	valid_0's multi_logloss: 8.80827	valid_1's multi_logloss: 3.18861
[9]	valid_0's multi_logloss: 8.50784	valid_1's multi_logloss: 3.21353
[10]	valid_0's multi_logloss: 8.22358	vali

In [37]:
# LightGBM GBDT with KFold or Stratified KFold
def lightgbm(params):
    
    print('\n--------------------------------------------\n')
    print('kfolded lightGBM\n')
    
    print('Train set shape:', dtrain.data.shape)
    print('Valid set shape:', dvalid.data.shape)

#     # Create arrays and dataframes to store results
#     oof_preds = np.zeros(df_train.shape[0])
#     sub_preds = np.zeros(df_test.shape[0])
#     df_feature_importance = pd.DataFrame()
    
    print('Number of features: {}'.format(len(features)))
                    
    def accuracy(preds, train_data):
        labels = train_data.get_label()
        pred = np.argmax(preds.reshape(n_classes, len(preds)//n_classes), axis=0)
        return 'accuracy', np.mean(labels == pred), True
    
    clf = lgb.train(
        params=params,
        train_set=dtrain,
#         num_boost_round=10000,
        valid_sets=[dtrain, dvalid],
        early_stopping_rounds=10,
        verbose_eval=True,
        feval=accuracy,
    )
    print('...model trained')

    train_preds = clf.predict(dtrain.data)
    valid_preds = clf.predict(dvalid.data)
    print('...predictions made')
    print('Accuracy train {:.6f}'.format(np.mean(np.argmax(train_preds, axis=1) == df_train['TARGET'].values)) )
    print('Accuracy valid {:.6f}'.format(np.mean(np.argmax(valid_preds, axis=1) == df_valid['TARGET'].values)) )
    return clf

params = {  "objective" : "multiclass",
            "num_class" : n_classes,
            'n_estimators': 10000,
            'learning_rate': .02,
            'num_leaves': 1000,
            'colsample_bytree': 1.,
            'subsample': 1.,
            'max_depth': 100,
            'reg_alpha': .041545473,
            'reg_lambda': .0735294,
            'min_split_gain': .0222415,
            'min_child_weight': 39.3259775,                
#             "device_type" : "gpu",
            "njobs" : os.cpu_count(),
}

clf = lightgbm(params)


--------------------------------------------

kfolded lightGBM

Train set shape: (393420, 128)
Valid set shape:

/jet/var/python/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


 (2016, 128)
Number of features: 128
[1]	training's multi_logloss: 6.94317	training's accuracy: 0.0142519	valid_1's multi_logloss: 5.68881	valid_1's accuracy: 0.121528
Training until validation scores don't improve for 10 rounds.
[2]	training's multi_logloss: 6.91078	training's accuracy: 0.0209649	valid_1's multi_logloss: 5.65548	valid_1's accuracy: 0.167163
[3]	training's multi_logloss: 6.87927	training's accuracy: 0.0262137	valid_1's multi_logloss: 5.62315	valid_1's accuracy: 0.176587
[4]	training's multi_logloss: 6.84904	training's accuracy: 0.0296985	valid_1's multi_logloss: 5.59527	valid_1's accuracy: 0.186012
[5]	training's multi_logloss: 6.8195	training's accuracy: 0.0320599	valid_1's multi_logloss: 5.56562	valid_1's accuracy: 0.189484
[6]	training's multi_logloss: 6.79073	training's accuracy: 0.0333715	valid_1's multi_logloss: 5.53707	valid_1's accuracy: 0.194444
[7]	training's multi_logloss: 6.76224	training's accuracy: 0.0348254	valid_1's multi_logloss: 5.50895	valid_1's accu

KeyboardInterrupt: 

In [ ]:
train_preds = clf.predict(dtrain.data)
valid_preds = clf.predict(dvalid.data)
train_preds.shape

In [32]:
np.argmax(train_preds, axis=1).shape

(39342,)